# Welcome! #
## Enzyme and Zygote Performance Shootout ##

### Project Information ###

If you're interested in seeing a project synopsis and the benchmarks for performance we'll measuring, please take a look at our project documentation. 

### What Now? ###

Let's start by setting up Enzyme and Zygote in our environment to start testing some differentiation. Our primary benchmark is the physical timing of a differentiation operation. In order to measure this we will simply be using the `@time` Julia method. These times will then be compared to measure how fast differentiation is actually done, along with comparing other important benchmarks.

#### Import the Packages ####

In [1]:
using Pkg

Pkg.add("Enzyme")
Pkg.add("Zygote")
Pkg.add("Plots")

In [2]:
using Zygote
using Enzyme
using Plots
using Statistics

#### What Next? ####

Now that the packages have been added to the environment, we can start testing them out. First, a quick demonstration of the timing function we will be using. 

In [3]:
function fib(n)
    if n <= 1
        return 1
    else
        return fib(n - 1) + fib(n - 2)
    end
end
    
@time fib(20)

  0.000036 seconds


10946

#### Functions ####

For this shootout we will be handling differentiation for rootfinding problems. Specifically, we will be testing differentiation efficiency for aiding five different rootfinding algorithms, being Halley's, Golbabai-Javidi, Newton's, Noor's, and Zhanlav's Method.

Those methods look like such:

**Halley's Method**

$x_{n + 1} = x_{n} - \frac{2f(x_n)f^{\prime}(x_n)}{2f^{\prime}(x_n)^2 - f(x_n)f^{\prime \prime}(x_n)}$

**Golbabai-Javidi Method**

$x_{n + 1} = x_{n} - \frac{f(x_n)}{f^{\prime}(x_n)} - \frac{f(x_n)f^{\prime \prime}(x_n)}{2(f^{\prime \prime \prime}(x_n) - f(x_n)f^{\prime}(x_n)f^{\prime \prime}(x_n))}$

**Newton's Method**

$x_{n + 1} = x_{n} - \frac{f(x_n)}{f^{\prime}(x_n)}$

**Noor's Method**

$y_n = x_n - \frac{f(x_n)}{f^{\prime}(x_n)}$

$x_{n + 1} = x_{n} - \frac{f(x_n)}{f^{\prime}(x_n)} + (\frac{f(x_n)}{f^{\prime}(x_n)})\frac{f^{\prime}(y_n)}{f^{\prime}(x_n)}$

**Zhanlav's Method**

$z_n = y_n - \frac{f(y_n)}{f^{\prime}(y_n)}$

$q_n = z_n - \frac{f(z_n)}{f^{\prime}(y_n)}$

$y_{n + 1} = z_n - \frac{f(z_n) + f(q_n)}{f^{\prime}(y_n)}$

Before implementation of these various rootfinding algorithms, however, we can demonstrate some fairly basic differentiation using some basic functions below. This, similar to the time test, will present the methods we will be using and how they work.

We will be performing both of these tests using Enzyme and Zygote on three different functions, which will be defined below.

#### Test Functions ####

$f(x) = 5x^{10}$

$g(x) = 3x^3(cos(x) - 10x)$

$h(x) = e^{\frac{5x}{2}(sin(x)^{e^x})}$

#### Zygote Method ####

Zygote is the auto differentiation tool specifically made for Julia and uses the `gradient` method for computing derivatives. We hypothesize that this tool will likely be better optimized for Julia, but we will test this below by timing some basic differentiation!

#### Enzyme Method ####

Enzyme is another auto differentiation tool that is not specifically designed for Julia and is more generally designed for many different languages. This package uses the `autodiff` method for computing derivatives. Given that this tool is not specifically built for Julia, we hypothesize that these method calls will be significantly less optimized.

#### What are we testing? ####

For all three functions we will be timing the differentiation speed of both packages at an x-value of $2$.

#### Function One ####

In [4]:
f(x) = 5x^10
x = 2
@time zyg = gradient(x -> f(x), x)
@time enz = autodiff(f, Active(x))
@show zyg
@show enz

 23.312653 seconds (28.55 M allocations: 1.644 GiB, 4.59% gc time, 100.01% compilation time)
 17.650693 seconds (19.74 M allocations: 1.105 GiB, 4.38% gc time, 0.03% compilation time)
zyg = (25600.0,)
enz = (25600.0,)


(25600.0,)

#### Function Two ####

In [5]:
gf(x) = 3x^3 * (cos(x) - 10x)
x = 2
@time zyg = gradient(x -> gf(x), x)
@time enz = autodiff(gf, Active(x))
@show zyg
@show enz

  0.356357 seconds (495.13 k allocations: 28.543 MiB, 29.38% gc time, 99.95% compilation time)
  4.130326 seconds (6.26 M allocations: 362.806 MiB, 3.88% gc time, 39.18% compilation time)
zyg = (-996.8044243595134,)
enz = (-996.8044243595134,)


(-996.8044243595134,)

#### Function Three ####

In [6]:
hf(x) = exp((5x / 2) * (sin(x)^(exp(x))))
x = 2
@time zyg = gradient(x -> hf(x), x)
@time enz = autodiff(hf, Active(x))
@show zyg
@show enz

  0.361814 seconds (649.20 k allocations: 40.221 MiB, 15.22% gc time, 99.96% compilation time)
  2.381995 seconds (3.32 M allocations: 188.232 MiB, 2.88% gc time, 91.09% compilation time)
zyg = (-105.63101149036947,)
enz = (-105.63101149036945,)


(-105.63101149036945,)

#### What are the results? ####

From using the differentiation method from Zygote and Enzyme in the above functions, we can see that both methods produce essentially the same results, being identical in the first two cases and different at the $10^{-14}$ digit in the last results. Reasonably, we can conclude that both of these methods are possess similar accuarcy from this small test. We can also see that both methods have relatively identical speeds, with Enzyme being generally a bit slower and Zygote a bit faster. In terms of time, no conclusions can really be drawn, so it might be worth performing these computations a few times and pulling the respective means, variances, standard deviations, and medians. Finally, we can clearly see that Enzyme also generally requires more memory allocation than Zygote and takes a smaller percentage of compilation time. This similarly is a bit hard to gage however which is better, so a similar approach to the aforementioned would be useful.

#### Data Science ####

Let's try collecting a larger sample set of differentiation tests to better understand which method seems to be more optimal. Below we have defined a for loop that calls the differentiation methods $100$ times each, and then collects the results. These results will be passed to a data science function at the end that uses a variety of Julia Statistics methods to compute the corresponding data science values. For the tests, we will once again differentiate at $x = 2$.

In [7]:
function dataSci(timeEnz, sizeEnz, timeZyg, sizeZyg)
    timeEnzMean = mean(timeEnz)
    sizeEnzMean = mean(sizeEnz)
    timeZygMean = mean(timeZyg)
    sizeZygMean = mean(sizeZyg)
    
    timeEnzVar = var(timeEnz)
    sizeEnzVar = var(sizeEnz)
    timeZygVar = var(timeZyg)
    sizeZygVar = var(sizeZyg)
    
    timeEnzStd = timeEnzVar ^ (1 / 2)
    sizeEnzStd = sizeEnzVar ^ (1 / 2)
    timeZygStd = timeZygVar ^ (1 / 2)
    sizeZygStd = sizeZygVar ^ (1 / 2)
    
    timeEnzMed = median(timeEnz)
    sizeEnzMed = median(sizeEnz)
    timeZygMed = median(timeZyg)
    sizeZygMed = median(sizeZyg)
    
    @show timeEnzMean
    @show timeEnzVar
    @show timeEnzStd
    @show timeEnzMed
    
    @show sizeEnzMean
    @show sizeEnzVar
    @show sizeEnzStd
    @show sizeEnzMed
    
    
    @show timeZygMean
    @show timeZygVar
    @show timeZygStd
    @show timeZygMed
    
    @show sizeZygMean
    @show sizeZygVar
    @show sizeZygStd
    @show sizeZygMed
end

dataSci (generic function with 1 method)

#### Function One ####

In [8]:
timesEnz1 = []
timesZyg1 = []
sizesEnz1 = []
sizesZyg1 = []

for i in 1:100
    
    f(x) = 5x^10 # I don't understand a ton about Julia, but if I define this outside of the loop,
                 # the timed method doesn't work???
    
    valZyg, tZyg, sizeZyg = @timed gradient(x -> f(x), x)

    push!(timesZyg1, tZyg)
    push!(sizesZyg1, sizeZyg)

    valEnz, tEnz, sizeEnz = @timed autodiff(f, Active(x))

    push!(timesEnz1, tEnz)
    push!(sizesEnz1, sizeEnz)

end

dataSci(timesEnz1, sizesEnz1, timesZyg1, sizesZyg1)

timeEnzMean = 0.06858078295
timeEnzVar = 0.00029176143945233626
timeEnzStd = 0.01708102571429293
timeEnzMed = 0.063657477
sizeEnzMean = 5.51844364e6
sizeEnzVar = 289297.6872727281
sizeEnzStd = 537.8640044404608
sizeEnzMed = 5.518323e6
timeZygMean = 0.03520859583999998
timeZygVar = 0.0001868100686014312
timeZygStd = 0.0136678479872082
timeZygMed = 0.0311291065
sizeZygMean = 3.64205973e6
sizeZygVar = 1.7174143826233292e9
sizeZygStd = 41441.698597226066
sizeZygMed = 3.637873e6


3.637873e6

#### Results Function One ####

- **Zygote**

    - **Time**
    
        - Mean = 0.04123 seconds
        - Variance = 0.000236 seconds
        - Standard Deviation = 0.01535 seconds
        - Median = 0.0366 seconds
       
    - **Alloc. Size**
    
        - Mean = 3,642,460 bytes (3.64 MB)
        - Variance = 1,713,173,490 bytes (1.71 GB)
        - Standard Deviation = 41,390 bytes (41.39 KB)
        - Median = 3,638,321 bytes (3.64 MB)

- **Enzyme**

    - **Time**
    
        - Mean = 0.09267 seconds
        - Variance = 0.002728 seconds
        - Standard Deviation = 0.05224 seconds
        - Median = 0.0836 seconds
    
    - **Alloc. Size**
    
        - Mean = 5,519,142 bytes (5.52 MB)
        - Variance = 49,098,291 bytes (49.10 MB)
        - Standard Deviation = 7,007.02 bytes (7.01 KB)
        - Median = 5,518,323 bytes (5.52 MB)

#### Function Two ####

In [9]:
timesEnz2 = []
timesZyg2 = []
sizesEnz2 = []
sizesZyg2 = []

for i in 1:100
    
    gf(x) = 3x^3 * (cos(x) - 10x) # I don't understand a ton about Julia, but if I define this outside of the loop,
                                             # the timed method doesn't work???
    
    valZyg, tZyg, sizeZyg = @timed gradient(x -> gf(x), x)

    push!(timesZyg2, tZyg)
    push!(sizesZyg2, sizeZyg)

    valEnz, tEnz, sizeEnz = @timed autodiff(gf, Active(x))

    push!(timesEnz2, tEnz)
    push!(sizesEnz2, sizeEnz)

end

dataSci(timesEnz2, sizesEnz2, timesZyg2, sizesZyg2)

timeEnzMean = 0.15323662477999997
timeEnzVar = 0.0006277404743735936
timeEnzStd = 0.025054749537235323
timeEnzMed = 0.14833649599999998
sizeEnzMean = 6.91565948e6
sizeEnzVar = 606044.8581818199
sizeEnzStd = 778.4888298375384
sizeEnzMed = 6.915419e6
timeZygMean = 0.05297227271
timeZygVar = 0.0012711410592914946
timeZygStd = 0.03565306521593192
timeZygMed = 0.0449094105
sizeZygMean = 6.02049362e6
sizeZygVar = 1.7153262670460622e9
sizeZygStd = 41416.49752267884
sizeZygMed = 6.016269e6


6.016269e6

#### Results Function Two ####

- **Zygote**

    - **Time**
    
        - Mean = 0.08564 seconds
        - Variance = 0.00988 seconds
        - Standard Deviation = 0.09940 seconds
        - Median = 0.05361 seconds
       
    - **Alloc. Size**
    
        - Mean = 6,022,823 bytes (6.02 MB)
        - Variance = 1,718,475,426 bytes (1.72 GB)
        - Standard Deviation = 41,454 bytes (41.45 KB)
        - Median = 6,018,635 bytes (6.02 MB)

- **Enzyme**

    - **Time**
   
        - Mean = 0.20543 seconds
        - Variance = 0.00845 seconds
        - Standard Deviation = 0.09191 seconds
        - Median = 0.17561 seconds
    
    - **Alloc. Size**
    
        - Mean = 6,915,644 bytes (6.91 MB)
        - Variance = 540,688 bytes (540.69 KB)
        - Standard Deviation = 735.31 bytes (735.31 B)
        - Median = 6,915,419 bytes (6.91 MB)

#### Function Three ####

In [10]:
timesEnz3 = []
timesZyg3 = []
sizesEnz3 = []
sizesZyg3 = []

for i in 1:100
    
    hf(x) = exp((5x / 2) * (sin(x)^(exp(x)))) # I don't understand a ton about Julia, but if I define this outside of the loop,
                                              # the timed method doesn't work???
    
    valZyg, tZyg, sizeZyg = @timed gradient(x -> hf(x), x)

    push!(timesZyg3, tZyg)
    push!(sizesZyg3, sizeZyg)

    valEnz, tEnz, sizeEnz = @timed autodiff(hf, Active(x))

    push!(timesEnz3, tEnz)
    push!(sizesEnz3, sizeEnz)

end

dataSci(timesEnz3, sizesEnz3, timesZyg3, sizesZyg3)

timeEnzMean = 0.1806698743000001
timeEnzVar = 0.0008602652893138969
timeEnzStd = 0.029330279393723763
timeEnzMed = 0.1744152195
sizeEnzMean = 8.28191524e6
sizeEnzVar = 1.3188319418181812e6
sizeEnzStd = 1148.4040847272274
sizeEnzMed = 8.281633e6
timeZygMean = 0.07138301557999999
timeZygVar = 0.0007141429585669801
timeZygStd = 0.026723453342840632
timeZygMed = 0.064054451
sizeZygMean = 6.94564253e6
sizeZygVar = 1.7102608380899878e9
sizeZygStd = 41355.29999999985
sizeZygMed = 6.941507e6


6.941507e6

#### Results Function Three ####

- **Zygote**

    - **Time**
    
        - Mean = 0.09241 seconds
        - Variance = 0.00484 seconds
        - Standard Deviation = 0.06958 seconds
        - Median = 0.07395 seconds
       
    - **Alloc. Size**
    
        - Mean = 6,945,055 (6.94 MB)
        - Variance = 1,710,260,838 bytes (1.71 GB)
        - Standard Deviation = 41,355 bytes (41.35 KB)
        - Median = 6,940,920 bytes (6.94 MB)

- **Enzyme**

    - **Time**
    
        - Mean = 0.22160 seconds
        - Variance = 0.00715 seconds
        - Standard Deviation = 0.08455 seconds
        - Median = 0.19926 seconds
    
    - **Alloc. Size**
    
        - Mean = 8,282,553 bytes (8.28 MB)
        - Variance = 49,230,194 bytes (49.23 MB)
        - Standard Deviation = 7,016.42 bytes (7.02 KB)
        - Median = 8,281,633 bytes (8.28 MB)

#### Implications? ####



#### Practical Application ####

Theory is great and all, but what's a simple application for differentiation? There's a wide range of applications from gradient-descent to modeling certain processes, but for our project we decided some simple rootfinding algorithms would be useful. Thus, below are some rootfinding implementations of these algorithms using Enzyme and Zygote which produce a more noticeable time and memory difference between the two packages.

# Newton's with Enzyme #

In [11]:

    # testing Newton's with Enzyme

function newtonE(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - fx / fpx
    end  
end

f(x) = cos(x) - x
newtonE(f, 1; tol=1e-15, verbose=true)

(0.7390851332151607, 0.0, 5)

# Newton's with Zygote #

In [12]:

    # testing Newton's with Zygote

function newtonZ(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Zygote.gradient(f, x))
        
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - fx / fpx
    end  
end

f(x) = cos(x) - x
newtonZ(f, 1; tol=1e-15, verbose=true)

(0.7390851332151607, 0.0, 5)

# Testing Newton's #

In [13]:
timesEnz = []
timesZyg = []
sizesEnz = []
sizesZyg = []

for i in 1:2
    
    func1() = newtonZ(f, 1; tol=1e-15, verbose=true)
    func2() = newtonE(f, 1; tol=1e-15, verbose=true)
    
    valZyg, tZyg, sizeZyg = @timed func1()

    push!(timesZyg, tZyg)
    push!(sizesZyg, sizeZyg)

    valEnz, tEnz, sizeEnz = @timed func2()

    push!(timesEnz, tEnz)
    push!(sizesEnz, sizeEnz)

end

dataSci(timesEnz, sizesEnz, timesZyg, sizesZyg)

timeEnzMean = 0.09380311549999999
timeEnzVar = 0.01759107669734898
timeEnzStd = 0.13263135638810675
timeEnzMed = 0.09380311549999999
sizeEnzMean = 7.1388415e6
sizeEnzVar = 1.019174352778125e14
sizeEnzStd = 1.0095416548008928e7
sizeEnzMed = 7.1388415e6
timeZygMean = 0.045284393500000006
timeZygVar = 0.004100671540322645
timeZygStd = 0.06403648600854552
timeZygMed = 0.045284393500000006
sizeZygMean = 6.751274e6
sizeZygVar = 9.1153352205e13
sizeZygStd = 9.547426470258884e6
sizeZygMed = 6.751274e6


6.751274e6

#### Results Newton's ####

- **Zygote**

    - **Time**  

        - Mean = 0.0402496005 seconds

    - **Alloc. Size**
    
        - Mean = 6.751258e6 (6.75 MB)

- **Enzyme**

    - **Time**

        - Mean = 0.108942394 seconds

    - **Alloc. Size**

        - Mean = 7.1389855e6 (7.1 MB)


# Halley's with Enzyme #

In [14]:

    # halley's method with Enzyme

function halleyE(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        fppx = first(Enzyme.autodiff(f, Active(fpx)))
        
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (2 * fx * fpx) / (2 * fpx^2 - fx * fppx)
    end  
end

f(x) = cos(x) - x
halleyE(f, 1; tol=1e-15, verbose=true)

(0.7390851332151607, 0.0, 5)

# Halley's with Zygote #

In [15]:

    # halley's method with Zygote

function halleyZ(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Zygote.gradient(f, x))
        fppx = first(Zygote.gradient(f, fpx))
        
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (2 * fx * fpx) / (2 * fpx^2 - fx * fppx)
    end  
end

f(x) = cos(x) - x
halleyZ(f, 1; tol=1e-15, verbose=true)

(0.7390851332151607, 0.0, 5)

# Testing Halley's #

In [16]:
timesEnz = []
timesZyg = []
sizesEnz = []
sizesZyg = []

for i in 1:2
    
    func1() = halleyZ(f, 1; tol=1e-15, verbose=true)
    func2() = halleyE(f, 1; tol=1e-15, verbose=true)
    
    valZyg, tZyg, sizeZyg = @timed func1()

    push!(timesZyg, tZyg)
    push!(sizesZyg, sizeZyg)

    valEnz, tEnz, sizeEnz = @timed func2()

    push!(timesEnz, tEnz)
    push!(sizesEnz, sizeEnz)

end

dataSci(timesEnz, sizesEnz, timesZyg, sizesZyg)

timeEnzMean = 0.09705718549999999
timeEnzVar = 0.018836933145854877
timeEnzStd = 0.1372477072517238
timeEnzMed = 0.09705718549999999
sizeEnzMean = 7.32284e6
sizeEnzVar = 1.07236723743872e14
sizeEnzStd = 1.0355516585080244e7
sizeEnzMed = 7.32284e6
timeZygMean = 0.0240745435
timeZygVar = 0.0011584752977961846
timeZygStd = 0.034036381972768266
timeZygMed = 0.0240745435
sizeZygMean = 6.77571e6
sizeZygVar = 9.1814421072392e13
sizeZygStd = 9.581984192869032e6
sizeZygMed = 6.77571e6


6.77571e6

#### Results Halley's ####

- **Zygote**

    - **Time**
    
        - Mean = 0.0296112535 seconds

    - **Alloc. Size**
    
        - Mean = 6.775326e6 (6.77 MB)

- **Enzyme**

    - **Time**

        - Mean = 0.107368892 seconds

    - **Alloc. Size**

        - Mean = 7.322984e6 (7.3 MB)



# Golbabai-Javidi's with Enzyme #

In [17]:

    # Golbabai-Javidi's method with Enzyme

function GJ_E(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        fppx = first(Enzyme.autodiff(f, Active(fpx)))
        fpppx = first(Enzyme.autodiff(f, Active(fppx)))
        
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (fx / fpx) - ((fx * fppx) / (2 * (fpppx - (fx * fpx * fppx))))
    end  
end

f(x) = cos(x) - x
GJ_E(f, 1; tol=1e-15, verbose=true)

(0.739085133215161, -6.661338147750939e-16, 8)

# Golbabai-Javidi's with Zygote #

In [18]:

    # Golbabai-Javidi's method with Zygote

function GJ_Z(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Zygote.gradient(f, x))
        fppx = first(Zygote.gradient(f, fpx))
        fpppx = first(Zygote.gradient(f, fppx))
        
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (fx / fpx) - ((fx * fppx) / (2 * (fpppx - (fx * fpx * fppx))))
    end  
end

f(x) = cos(x) - x
GJ_Z(f, 1; tol=1e-15, verbose=true)

(0.739085133215161, -6.661338147750939e-16, 8)

# Testing Golbabai-Javidi's #

In [19]:
timesEnz = []
timesZyg = []
sizesEnz = []
sizesZyg = []

for i in 1:2
    
    func1() = GJ_Z(f, 1; tol=1e-15, verbose=true)
    func2() = GJ_E(f, 1; tol=1e-15, verbose=true)
    
    valZyg, tZyg, sizeZyg = @timed func1()

    push!(timesZyg, tZyg)
    push!(sizesZyg, sizeZyg)

    valEnz, tEnz, sizeEnz = @timed func2()

    push!(timesEnz, tEnz)
    push!(sizesEnz, sizeEnz)

end

dataSci(timesEnz, sizesEnz, timesZyg, sizesZyg)

timeEnzMean = 0.10511048199999999
timeEnzVar = 0.022089704497476128
timeEnzStd = 0.14862605591711073
timeEnzMed = 0.10511048199999999
sizeEnzMean = 7.4284765e6
sizeEnzVar = 1.103464609065845e14
sizeEnzStd = 1.0504592372223899e7
sizeEnzMed = 7.4284765e6
timeZygMean = 0.025311380499999998
timeZygVar = 0.0012808844994971202
timeZygStd = 0.0357894467615402
timeZygMed = 0.025311380499999998
sizeZygMean = 6.805606e6
sizeZygVar = 9.2626448331848e13
sizeZygStd = 9.624263521529738e6
sizeZygMed = 6.805606e6


6.805606e6

#### Results Golbabai-Javidi's ####

- **Zygote**

    - **Time**
    
        - Mean = 0.072431259 seconds

    - **Alloc. Size**
    
        - Mean = 6.805222e6 (6.8 MB)

- **Enzyme**

    - **Time**

        - Mean = 0.119032858 seconds

    - **Alloc. Size**

        - Mean = 7.4286205e6 (7.4 MB)




# Noor's with Enzyme #

In [20]:

    # Noor's method with Enzyme

function noorE(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        y = x - fx/fpx
        fpy = first(Enzyme.autodiff(f, Active(y)))
        
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (fx / fpx) + (fx / fpx) * (fpy / fpx)
    end  
end

f(x) = x * x
noorE(f, 1; tol=1e-15, verbose=true)

(2.391867219711847e-8, 5.72102879673208e-16, 62)

# Noor's with Zygote #

In [21]:

    # Noor's method with Zygote

function noorZ(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Zygote.gradient(f, x))
        y = x - fx/fpx
        fpy = first(Zygote.gradient(f, y))
        
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (fx / fpx) + (fx / fpx) * (fpy / fpx)
    end  
end

f(x) = x * x
noorZ(f, 1; tol=1e-15, verbose=true)

(2.391867219711847e-8, 5.72102879673208e-16, 62)

# Testing Noor's #

In [22]:
timesEnz = []
timesZyg = []
sizesEnz = []
sizesZyg = []

for i in 1:2
    
    func1() = noorZ(f, 1; tol=1e-15, verbose=true)
    func2() = noorE(f, 1; tol=1e-15, verbose=true)
    
    valZyg, tZyg, sizeZyg = @timed func1()

    push!(timesZyg, tZyg)
    push!(sizesZyg, sizeZyg)

    valEnz, tEnz, sizeEnz = @timed func2()

    push!(timesEnz, tEnz)
    push!(sizesEnz, sizeEnz)

end

dataSci(timesEnz, sizesEnz, timesZyg, sizesZyg)

timeEnzMean = 0.0565241755
timeEnzVar = 0.006373006862040013
timeEnzStd = 0.07983111462356024
timeEnzMed = 0.0565241755
sizeEnzMean = 5.5767465e6
sizeEnzVar = 6.21509589759645e13
sizeEnzStd = 7.88358795067097e6
sizeEnzMed = 5.5767465e6
timeZygMean = 0.0283539205
timeZygVar = 0.0016073063714890127
timeZygStd = 0.040091225617197246
timeZygMed = 0.0283539205
sizeZygMean = 5.24399e6
sizeZygVar = 5.4994163725512e13
sizeZygStd = 7.415804995110915e6
sizeZygMed = 5.24399e6


5.24399e6

#### Results Noor's ####

- **Zygote**

    - **Time**
    
        - Mean = 0.0273126905 seconds

    - **Alloc. Size**
    
        - Mean = 5.243606e6 (5.2 MB)

- **Enzyme**

    - **Time**

        - Mean = 0.0673777475 seconds

    - **Alloc. Size**

        - Mean = 5.5768905e6 (5.6 MB)


# Zhanlav's with Enzyme #

In [23]:

    # Zhanlav's method with Enzyme

function zhanlavE(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:1000 # max number of iterations
        
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        
        z = x - fx/fpx
        fz = f(z)
        fpz = first(Enzyme.autodiff(f, Active(z)))

        q = z - fz/fpz
        fq = f(q)
        
        
        if abs(fx) < tol
            return x, fx, k
        end
        x = z - (fz + fq)/fpx
    end  
end

f(x) = cos(x) - x
zhanlavE(f, 1; tol=1e-15, verbose=true)

(0.7390851332151607, 0.0, 4)

# Zhanlav's with Zygote #

In [24]:

    # Zhanlav's method with Zygote

function zhanlavZ(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:1000 # max number of iterations
        
        fx = f(x)
        fpx = first(Zygote.gradient(f, x))
        
        z = x - fx/fpx
        fz = f(z)
        fpz = first(Zygote.gradient(f, z))
        
        q = z - fz/fpz
        fq = f(q)
        
        if abs(fx) < tol
            return x, fx, k
        end
        x = z - (fz + fq)/fpx
    end  
end

f(x) = cos(x) - x
zhanlavZ(f, 1; tol=1e-15, verbose=true)

(0.7390851332151607, 0.0, 4)

# Testing Zhanlav's #

In [25]:
timesEnz = []
timesZyg = []
sizesEnz = []
sizesZyg = []

for i in 1:2
    
    func1() = zhanlavZ(f, 1; tol=1e-15, verbose=true)
    func2() = zhanlavE(f, 1; tol=1e-15, verbose=true)
    
    valZyg, tZyg, sizeZyg = @timed func1()

    push!(timesZyg, tZyg)
    push!(sizesZyg, sizeZyg)

    valEnz, tEnz, sizeEnz = @timed func2()

    push!(timesEnz, tEnz)
    push!(sizesEnz, sizeEnz)

end

dataSci(timesEnz, sizesEnz, timesZyg, sizesZyg)

timeEnzMean = 0.1182120215
timeEnzVar = 0.027944654685910796
timeEnzStd = 0.16716654774777995
timeEnzMed = 0.1182120215
sizeEnzMean = 7.3462295e6
sizeEnzVar = 1.079238324900125e14
sizeEnzStd = 1.0388639588031365e7
sizeEnzMed = 7.3462295e6
timeZygMean = 0.023921065
timeZygVar = 0.001143907350591618
timeZygStd = 0.03382169940425256
timeZygMed = 0.023921065
sizeZygMean = 6.786822e6
sizeZygVar = 9.2115824827208e13
sizeZygStd = 9.597698933974123e6
sizeZygMed = 6.786822e6


6.786822e6

#### Results Zhanlav's ####

- **Zygote**

    - **Time**
    
        - Mean = 0.0455876415 seconds

    - **Alloc. Size**
    
        - Mean = 6.786438e6 (6.8 MB)

- **Enzyme**

    - **Time**

        - Mean = 0.206026736 seconds

    - **Alloc. Size**

        - Mean = 7.3463735e6 (7.3 MB)

# Root-finding Results #

For every method we have that Zygote is quicker and allocates less memory. In some cases Enzyme was much slower, but the memory allocation was generally the same.